# Datawrapper Tutorial

Datawrapper ist ein service um ansprechende, informative und einfach zu nutzende Visualisierungen zu erstellen. In diesem Tutorial werden einige Beispiele gezeigt um Visualisierungen mit Datawrapper zu erstellen.

Für dieses Tutorial werden die Python Pakete `datawrapper`, `requests` und `pandas` benötigt, welche mit `pip` installiert werden können.

In [10]:
!pip --quiet install datawrapper requests pandas

Datawrapper erfordert einen API Access Token, welcher für eingeloggte Benutzer auf der Webseite unter Settings->API-Tokens erstellt werden kann. Für die folgenden Beispiele werden die Berechtigungen Chart read, write und Visualization read benötigt.

In [5]:
from getpass import getpass
access_token = getpass("Bitte Access Token eingeben: ") 

Bitte Access Token eingeben:  ································································


In [6]:
from datawrapper import Datawrapper
dw = Datawrapper(access_token = access_token)

In [12]:
import requests
import pandas as pd

antwort = requests.get('https://www.openligadb.de/api/getmatchdata/bl1/2020')
spiele = pd.DataFrame(antwort.json())

In [19]:
spiele.iloc[0]

MatchID                                                           58577
MatchDateTime                                       2020-09-18T20:30:00
TimeZoneID                                      W. Europe Standard Time
LeagueId                                                           4442
LeagueName                              1. Fußball-Bundesliga 2020/2021
MatchDateTimeUTC                                   2020-09-18T18:30:00Z
Group                 {'GroupName': '1. Spieltag', 'GroupOrderID': 1...
Team1                 {'TeamId': 40, 'TeamName': 'FC Bayern München'...
Team2                 {'TeamId': 9, 'TeamName': 'FC Schalke 04', 'Sh...
LastUpdateDateTime                              2020-09-20T16:15:35.617
MatchIsFinished                                                    True
MatchResults          [{'ResultID': 92144, 'ResultName': 'Endergebni...
Goals                 [{'GoalID': 84638, 'ScoreTeam1': 1, 'ScoreTeam...
Location              {'LocationID': 34, 'LocationCity': 'Münche

IndexError: list index out of range